In [99]:
import tensorflow as tf
import pandas as pd
import numpy as np
import sklearn as sk
import timeit
import matplotlib.pyplot as plt
from sklearn.svm import SVC
import math

In [2]:
def reshpe(X):
    numpy = X.as_matrix()
    shape = X.shape[0]
    numpy = numpy.reshape(shape,28,28,1)
    return numpy
def get_data(num_training=41000, num_validation = 1000):
    X_train = pd.read_csv('../Digit/Digits/train.csv')
    Y_train =X_train['label']
    Y_train =Y_train.as_matrix()
    del X_train['label']
    X_test  = pd.read_csv(r'../Digit/Digits/test.csv')
    X_test = reshpe(X_test)
    mask = range(num_training, num_training+num_validation)
    X_val = X_train.iloc[mask]
    X_val = reshpe(X_val)
    Y_val = Y_train[mask]
    mask= range(num_training)
    X_train = X_train.iloc[mask]
    X_train = reshpe(X_train)
    Y_train = Y_train[mask]
    return X_train,Y_train,X_val,Y_val,X_test
X_train,Y_train,X_val,Y_val,X_test=get_data()


In [3]:
X_train.shape

(41000, 28, 28, 1)

In [4]:
def run_model(session, predict, loss_val, Xd, yd,
              epochs=1, batch_size=64, print_every=100,
              training=None, plot_losses=False):
    # have tensorflow compute accuracy
    correct_prediction = tf.equal(tf.argmax(predict,1), y)
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
    # shuffle indicies
    train_indicies = np.arange(Xd.shape[0])
    np.random.shuffle(train_indicies)

    training_now = training is not None
    
    # setting up variables we want to compute (and optimizing)
    # if we have a training function, add that to things we compute
    variables = [mean_loss,correct_prediction,accuracy]
    if training_now:
        variables[-1] = training
    
    # counter 
    iter_cnt = 0
    for e in range(epochs):
        # keep track of losses and accuracy
        correct = 0
        losses = []
        # make sure we iterate over the dataset once
        for i in range(int(math.ceil(Xd.shape[0]/batch_size))):
            # generate indicies for the batch
            start_idx = (i*batch_size)%Xd.shape[0]
            idx = train_indicies[start_idx:start_idx+batch_size]
            
            # create a feed dictionary for this batch
            feed_dict = {X: Xd[idx,:],
                         y: yd[idx],
                         is_training: training_now }
            # get batch size
            actual_batch_size = yd[idx].shape[0]
            
            # have tensorflow compute loss and correct predictions
            # and (if given) perform a training step
            loss, corr, _ = session.run(variables,feed_dict=feed_dict)
            
            # aggregate performance stats
            losses.append(loss*actual_batch_size)
            correct += np.sum(corr)
            
            # print every now and then
            if training_now and (iter_cnt % print_every) == 0:
                print("Iteration {0}: with minibatch training loss = {1:.3g} and accuracy of {2:.2g}"\
                      .format(iter_cnt,loss,np.sum(corr)/actual_batch_size))
            iter_cnt += 1
        total_correct = correct/Xd.shape[0]
        total_loss = np.sum(losses)/Xd.shape[0]
        print("Epoch {2}, Overall loss = {0:.3g} and accuracy of {1:.3g}"\
              .format(total_loss,total_correct,e+1))
        if plot_losses:
            plt.plot(losses)
            plt.grid(True)
            plt.title('Epoch {} Loss'.format(e+1))
            plt.xlabel('minibatch number')
            plt.ylabel('minibatch loss')
            plt.show()
    return total_loss,total_correct

In [259]:
tf.reset_default_graph()

# define our input (e.g. the data that changes every batch)
# The first dim is None, and gets sets automatically based on batch size fed in
X = tf.placeholder(tf.float32, [None, 28, 28, 1])
y = tf.placeholder(tf.int64, [None])
is_training = tf.placeholder(tf.bool)

# define model
def complex_model(X,y,is_training):
    # setup variables
    Wconv1 = tf.get_variable("Wconv1", shape=[7, 7, 1,28])
    bconv1 = tf.get_variable("bconv1", shape=[28])
    Wconv2 = tf.get_variable("Wconv2", shape=[7, 7, 28,56])
    bconv2 = tf.get_variable("bconv2", shape=[56])
    Wconv3 = tf.get_variable("Wconv3", shape=[7, 7, 56,56*2])
    bconv3 = tf.get_variable("bconv3", shape=[56*2])    
    gamma = tf.get_variable("gamma", shape=[4096])
    beta = tf.get_variable("beta", shape=[4096])
    gamma2 = tf.get_variable("gamma2", shape=[4096])
    beta2 = tf.get_variable("beta2", shape=[4096])
    W_affine1 = tf.get_variable("W_affine1", shape=[4*4*56*2, 4096])
    b_affine1 = tf.get_variable("b_affine1", shape=[4096])
    W_affine3 = tf.get_variable("W_affine3", shape=[4096, 4096])
    b_affine3 = tf.get_variable("b_affine3", shape=[4096])
    W_affine2 = tf.get_variable("W_affine2", shape=[4096, 10])
    b_affine2 = tf.get_variable("b_affine2", shape=[10])
    
    a1 = tf.nn.conv2d(X, Wconv1, strides=[1,1,1,1], padding='SAME') + bconv1
    h1 = tf.nn.relu(a1)
    max_pool_out1 = tf.nn.max_pool(h1, ksize=[1, 2, 2, 1], strides=[1,2,2,1], padding='SAME')
    a2 = tf.nn.conv2d(max_pool_out1, Wconv2, strides=[1,1,1,1], padding='SAME') + bconv2
    h2 = tf.nn.relu(a2)
    max_pool_out2 = tf.nn.max_pool(h2, ksize=[1, 2, 2, 1], strides=[1,2,2,1], padding='SAME')
    a3 = tf.nn.conv2d(max_pool_out2, Wconv3, strides=[1,1,1,1], padding='SAME') + bconv3
    h3 = tf.nn.relu(a3)
    max_pool_out3 = tf.nn.max_pool(h3, ksize=[1, 2, 2, 1], strides=[1,2,2,1], padding='SAME')
    max_pool_out3 = tf.reshape(max_pool_out3,[-1,4*4*56*2])
    affine_out1 = tf.matmul(max_pool_out3,W_affine1) + b_affine1
    #mean,var = tf.nn.moments(affine_out1,axes=[0], keep_dims=True)
    #batch_out = tf.nn.batch_normalization(affine_out1,mean,var,beta,gamma,variance_epsilon=1e-7)
    h4 = tf.nn.relu(affine_out1)
    affine_out3 = tf.matmul(h4,W_affine3) + b_affine3
    mean2,var2 = tf.nn.moments(affine_out3,axes=[0], keep_dims=True)
    batch_out2 = tf.nn.batch_normalization(affine_out3,mean2,var2,beta2,gamma2,variance_epsilon=1e-7)
    h2 = tf.nn.relu(batch_out2)
    affine_out2 = tf.matmul(h2,W_affine2) + b_affine2
    return affine_out2

y_out = complex_model(X,y,is_training)

In [260]:
total_loss = tf.losses.sigmoid_cross_entropy(tf.one_hot(y,10),logits=y_out)
mean_loss = tf.reduce_mean(total_loss)
optimizer = tf.train.AdamOptimizer(0.001)
train_step = optimizer.minimize(mean_loss)

In [261]:
extra_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
with tf.control_dependencies(extra_update_ops):
    train_step = optimizer.minimize(mean_loss)

In [ ]:
sess = tf.Session()

sess.run(tf.global_variables_initializer())
print('Training')
run_model(sess,y_out,mean_loss,X_train,Y_train,2,64,100,train_step)

Training
Iteration 0: with minibatch training loss = 0.704 and accuracy of 0.078


In [ ]:
print('Validation')
run_model(sess,y_out,mean_loss,X_val,Y_val,1,256)

In [244]:
predict = sess.run(y_out,feed_dict={X:X_test})

In [228]:
print(predict[1])


[ 5.42539358 -6.36009979 -6.64676952 -7.58563423 -6.54423428 -6.94392538
 -7.33768511 -7.84318018 -6.32241201 -8.01520348]


In [231]:
final_ans = np.argmax(predict, axis=1)
final_ans=final_ans.astype(int)

In [232]:
np.savetxt("predictions.csv", final_ans, delimiter=",")